In [ ]:
from google.colab import drive
import os
drive.mount('/content/gdrive')
# Establecer ruta de acceso en dr
import os
print(os.getcwd())
os.chdir("/content/gdrive/My Drive")

Mounted at /content/gdrive
/content


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import xgboost as xgb#emsamble bosque de bosque de decision
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.model_selection import HalvingRandomSearchCV

**Datos**

https://www.kaggle.com/datasets/mlg-ulb/creditcardfraudv


**Contexto**

Es importante que las compañías de tarjetas de crédito puedan reconocer las transacciones de tarjetas de crédito fraudulentas para que a los clientes no se les cobre por artículos que no compraron.

**Contenido**

El conjunto de datos contiene transacciones realizadas con tarjetas de crédito en septiembre de 2013 por titulares de tarjetas europeos.

Este conjunto de datos presenta transacciones que ocurrieron en dos días, donde tenemos 492 fraudes de 284,807 transacciones. El conjunto de datos está muy desequilibrado, la clase positiva (fraudes) representa el 0,172 % de todas las transacciones.

Contiene solo variables de entrada numéricas que son el resultado de una transformación PCA.

Desafortunadamente, debido a problemas de confidencialidad, no se pueden proporcionar las características originales ni más información general sobre los datos. Las características V1, V2, … V28 son los principales componentes obtenidos con PCA, las únicas características que no han sido transformadas con PCA son **Time y Amount**.


La característica **Time** contiene los segundos transcurridos entre cada transacción y la primera transacción en el conjunto de datos. La variable **Amount** es la cantidad de la transacción, esta función se puede utilizar para el aprendizaje sensible a los costos dependiente del ejemplo. **Class** es la variable de respuesta y toma valor 1 en caso de fraude y 0 en caso contrario.

In [2]:
# Lectura de DF
df=pd.read_csv('/content/creditcard.csv')
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [3]:
# Vamos a eliminar la columna tiempo
df= df.drop(columns='Time')
# Estandarizamos la columna Amount
df['Amount']=(df['Amount']- np.mean(df['Amount']))/np.std(df.Amount)
df.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,0.244964,0
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,-0.342475,0
2,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,1.160686,0
3,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,0.140534,0
4,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,-0.073403,0


In [4]:
df['Class'].value_counts()#clasificacion

0    284315
1       492
Name: Class, dtype: int64

In [5]:
# El problema es muy desbalanceado tranajaremos con una muestra para resolver el problema
df_ones=df[df['Class']==1] # Filtro de caracteristica
print(df_ones.shape)
df_ones.head()

(492, 30)


,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
541,-2.312227,1.951992,-1.609851,3.997906,-0.522188,-1.426545,-2.537387,1.391657,-2.770089,-2.772272,...,0.517232,-0.035049,-0.465211,0.320198,0.044519,0.177840,0.261145,-0.143276,-0.353229,1
623,-3.043541,-3.157307,1.088463,2.288644,1.359805,-1.064823,0.325574,-0.067794,-0.270953,-0.838587,...,0.661696,0.435477,1.375966,-0.293803,0.279798,-0.145362,-0.252773,0.035764,1.761758,1
4920,-2.303350,1.759247,-0.359745,2.330243,-0.821628,-0.075788,0.562320,-0.399147,-0.238253,-1.525412,...,-0.294166,-0.932391,0.172726,-0.087330,-0.156114,-0.542628,0.039566,-0.153029,0.606031,1
6108,-4.397974,1.358367,-2.592844,2.679787,-1.128131,-1.706536,-3.496197,-0.248778,-0.247768,-4.801637,...,0.573574,0.176968,-0.436207,-0.053502,0.252405,-0.657488,-0.827136,0.849573,-0.117342,1
6329,1.234235,3.019740,-4.304597,4.732795,3.624201,-1.357746,1.713445,-0.496358,-1.282858,-2.447469,...,-0.379068,-0.704181,-0.656805,-1.632653,1.488901,0.566797,-0.010016,0.146793,-0.349231,1


In [6]:
df_zeros=df[df['Class']==0] # Filtro de NO caracteristica
df_zeros= df_zeros.sample(3*df_ones.shape[0]) # Tamaño de muestra 3 veces el de la caracteristica
print(df_zeros.shape)

(1476, 30)


In [7]:
# Concatenar
df_final=pd.DataFrame(np.concatenate([df_ones, df_zeros],axis=0), columns=df.columns)
print(df_final.shape)
df_final.head()

(1968, 30)


,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,-2.312227,1.951992,-1.609851,3.997906,-0.522188,-1.426545,-2.537387,1.391657,-2.770089,-2.772272,...,0.517232,-0.035049,-0.465211,0.320198,0.044519,0.177840,0.261145,-0.143276,-0.353229,1.0
1,-3.043541,-3.157307,1.088463,2.288644,1.359805,-1.064823,0.325574,-0.067794,-0.270953,-0.838587,...,0.661696,0.435477,1.375966,-0.293803,0.279798,-0.145362,-0.252773,0.035764,1.761758,1.0
2,-2.303350,1.759247,-0.359745,2.330243,-0.821628,-0.075788,0.562320,-0.399147,-0.238253,-1.525412,...,-0.294166,-0.932391,0.172726,-0.087330,-0.156114,-0.542628,0.039566,-0.153029,0.606031,1.0
3,-4.397974,1.358367,-2.592844,2.679787,-1.128131,-1.706536,-3.496197,-0.248778,-0.247768,-4.801637,...,0.573574,0.176968,-0.436207,-0.053502,0.252405,-0.657488,-0.827136,0.849573,-0.117342,1.0
4,1.234235,3.019740,-4.304597,4.732795,3.624201,-1.357746,1.713445,-0.496358,-1.282858,-2.447469,...,-0.379068,-0.704181,-0.656805,-1.632653,1.488901,0.566797,-0.010016,0.146793,-0.349231,1.0


In [8]:
df_final.isnull().sum()

V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [9]:
# Separar en X y y
y= df_final.Class
X= df_final.drop(columns='Class', axis=1)
print(X.shape, y.shape)

(1968, 29) (1968,)


In [10]:
# Separar en train y test
X_train,X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
print(X_train.shape, X_test.shape)

(1377, 29) (591, 29)


# Seleccion manual

In [15]:
xgb.XGBClassifier?

In [13]:
model= xgb.XGBClassifier(learning_rate=0.001)#mdoelo
# Lista de hiperparametros
params_1 = {'criterion': 'gini', 'splitter': 'best', 'max_depth': 5}
params_2 = {'criterion': 'entropy', 'splitter': 'random', 'max_depth': 7}
params_3 = {'criterion': 'gini', 'splitter': 'random', 'max_depth': 10}

## Estos procesos pueden demorar bastante !!

In [14]:
# Modelo 1
model.set_params(**params_1).fit(X_train, y_train)
print(f'Accuracy para Modelo 1 = {round(accuracy_score(y_test, model.predict(X_test)), 5)}')
# Modelo 2
model.set_params(**params_2).fit(X_train, y_train)
print(f'Accuracy para Modelo 2 = {round(accuracy_score(y_test, model.predict(X_test)), 5)}')
# Modelo 3
model.set_params(**params_3).fit(X_train, y_train)
print(f'Accuracy para Modelo 3 = {round(accuracy_score(y_test, model.predict(X_test)), 5)}')

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [00:33:20] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "criterion", "splitter" } are not used.

  warnings.warn(smsg, UserWarning)


Accuracy para Modelo 1 = 0.74619


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [00:33:20] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "criterion", "splitter" } are not used.

  warnings.warn(smsg, UserWarning)


Accuracy para Modelo 2 = 0.74619


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [00:33:21] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "criterion", "splitter" } are not used.

  warnings.warn(smsg, UserWarning)


Accuracy para Modelo 3 = 0.74619


# Grid Search

In [18]:
#xbgboost
params_grid = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'max_depth': [5,6,7],
        'criterion':['entropy','gini']
        }

In [19]:
# tiempo de ejecucuon 424 seg
#cv=StratifiedKFold()
grid_cv = GridSearchCV(model, params_grid, scoring="accuracy", n_jobs=-1, cv=3)#clasificacion||regresion=r2
grid_cv.fit(X_train, y_train)

print("Mejores Parametros", grid_cv.best_params_)#mejor combinacion de hiperparametros
print("Mejor CV score", grid_cv.best_score_)#cual es mi mejor score con esa combinacion train vs validacion
#OJO CON QUE EL SCORE DE TRAIN VS LA VALIDACION SEA MAS ALTO DEL QUE NOS ARROJE EL TRAIN VS TEST===OVERFITTING
print(f'Accuracy del modelo = {round(accuracy_score(y_test, grid_cv.predict(X_test)), 5)}')#train vs test

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [01:00:42] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "criterion", "splitter" } are not used.

  warnings.warn(smsg, UserWarning)


Mejores Parametros {'criterion': 'entropy', 'gamma': 0.5, 'max_depth': 5, 'min_child_weight': 1}
Mejor CV score 0.7516339869281046
Accuracy del modelo = 0.74619


# Randomized Search CV

In [20]:
# tiempo de ejecucuon 7 seg
grid_cv = RandomizedSearchCV(model, params_grid, scoring="accuracy", n_jobs=-1, cv=3)#FOLD 3
grid_cv.fit(X_train, y_train)

print("Mejores parametros", grid_cv.best_params_)
print("Mejor score de CV", grid_cv.best_score_)
print(f'Accuracy del modelo = {round(accuracy_score(y_test, grid_cv.predict(X_test)), 5)}')

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [01:05:15] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "criterion", "splitter" } are not used.

  warnings.warn(smsg, UserWarning)


Mejores parametros {'min_child_weight': 10, 'max_depth': 7, 'gamma': 2, 'criterion': 'entropy'}
Mejor score de CV 0.7516339869281046
Accuracy del modelo = 0.74619


# Halving Grid Search

In [21]:
# tiempo de ejecucuon 125 seg
halving_cv = HalvingGridSearchCV(model, params_grid, scoring="accuracy", factor=3)#20 X 3=60    ->100
halving_cv.fit(X_train, y_train)

print("Mejores parametros", halving_cv.best_params_)
print("Mejor Score CV", halving_cv.best_score_)
print(f'Accuracy del modelo = {round(accuracy_score(y_test, halving_cv.predict(X_test)), 5)}')

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [01:11:57] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "criterion", "splitter" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [01:11:57] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "criterion", "splitter" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [01:11:57] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "criterion", "splitter" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [01:11:57] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "criterion", "splitter" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [01:11:57] WARNING: /workspace/src/learner.cc:742: 
Parameters

Mejores parametros {'criterion': 'entropy', 'gamma': 0.5, 'max_depth': 6, 'min_child_weight': 1}
Mejor Score CV 0.7430252682589131
Accuracy del modelo = 0.74619


# Halving Randomized Search

In [22]:
# tiempo de ejecucuon 13 seg
halving_cv = HalvingRandomSearchCV(model, params_grid, scoring="accuracy", factor=3)
halving_cv.fit(X_train, y_train)

print("Mejores parametros", halving_cv.best_params_)
print("Mejor CV score", halving_cv.best_score_)
print(f'Accuracy del modelo = {round(accuracy_score(y_test, halving_cv.predict(X_test)), 5)}')

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [01:13:40] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "criterion", "splitter" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [01:13:40] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "criterion", "splitter" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [01:13:40] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "criterion", "splitter" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [01:13:40] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "criterion", "splitter" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [01:13:40] WARNING: /workspace/src/learner.cc:742: 
Parameters

Mejores parametros {'min_child_weight': 1, 'max_depth': 5, 'gamma': 0.5, 'criterion': 'gini'}
Mejor CV score 0.7485981308411216
Accuracy del modelo = 0.74619


https://analyticsindiamag.com/how-to-use-hyperopt-for-distributed-hyperparameter-optimisation/

https://towardsdatascience.com/a-guide-to-find-the-best-boosting-model-using-bayesian-hyperparameter-tuning-but-without-c98b6a1ecac8